In [1]:
#List of Steps to get the pcd files generated from the point cliud 
import os
import shutil
import numpy as np
import struct
import numpy as np 
import time
import open3d
import pandas as pd 
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [15]:
imageFileList  = os.listdir(r'C:\Users\shyam\OneDrive\Pictures\KITTI')
print(len(imageFileList))

53


In [ ]:
folderPath = r'D:\3rd Semester\Machine Vision\Project\training\data_object_velodyne\training\velodyne'
destination = r'C:\Users\shyam\OneDrive\Pictures\KITTIBIN'

KittiDataset = os.listdir(folderPath)

for filename in KittiDataset:
    for binfiles in imageFileList:
        binfiles = os.path.splitext(binfiles)[0]+".bin" 
        file = os.path.join(folderPath,binfiles)
        if binfiles in filename:
            print(file)
            shutil.copy(file,destination)
        else:
             pass

In [ ]:
def generate_pcd(file_to_open,file_to_save):
    size_float = 4
    list_pcd = []
    with open (file_to_open, "rb") as f:
        byte = f.read(size_float*4)
        while byte:
            x,y,z,intensity = struct.unpack("ffff", byte)
            list_pcd.append([x, y, z])
            byte = f.read(size_float*4)
    np_pcd = np.asarray(list_pcd)
    pcd = open3d.geometry.PointCloud()
    v3d = open3d.utility.Vector3dVector
    pcd.points = v3d(np_pcd)
    open3d.io.write_point_cloud(file_to_save, pcd)

In [ ]:
source = r'C:\Users\shyam\OneDrive\Pictures\KITTIBIN' 
dest = r'C:\Users\shyam\OneDrive\Pictures\KITTIPCD'
binfiles = r'C:\Users\shyam\OneDrive\Pictures\KITTIBIN'
for binfiles in os.listdir(binfiles):
    pointClouds = os.path.splitext(binfiles)[0]+".pcd"
    files_to_open = os.path.join(source,binfiles)
    files_to_save = os.path.join(dest,pointClouds)
    print(files_to_save)
    generate_pcd(files_to_open,files_to_save)
    

In [2]:
pcd = open3d.io.read_point_cloud(r'C:\Users\shyam\OneDrive\Pictures\KITTIPCD\000339.pcd')

open3d.visualization.draw_geometries(([pcd]))
print(len(pcd.points))

115777


In [104]:
#Step 2 : Voxel Downsampling:
print('before downsampling:' , len(pcd.points))
downsample = pcd.voxel_down_sample(voxel_size=0.05)
print('fter Voxel downsampling :', len(downsample.points))
# open3d.visualization.draw_geometries([downsample])

Point clouds before Voxel downsampling : 113272
points after Voxel downsampling : 78817


In [105]:
#Segmentation : RANSAC

plane_model, inliers = downsample.segment_plane(distance_threshold=0.3,
                                      ransac_n=3,
                                      num_iterations=250)
inlier_cloud=downsample.select_by_index(inliers)
outlier_cloud=downsample.select_by_index(inliers,invert=True)
inlier_cloud.paint_uniform_color([1,0,0])#Inliers - Red
outlier_cloud.paint_uniform_color([0,0,1])# Outliers - Blue
open3d.visualization.draw_geometries([inlier_cloud,outlier_cloud])



time consumed 2.2794361114501953


In [106]:
# Clustering - DBScan
with open3d.utility.VerbosityContextManager(open3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(outlier_cloud.cluster_dbscan(eps=0.5,min_points=5,print_progress=True))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels/(max_label if max_label > 0 else 1))
colors[labels < 0] = 0
outlier_cloud.colors = open3d.utility.Vector3dVector(colors[:,:3])
# open3d.visualization.draw_geometries([inlier_cloud,outlier_cloud])

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 296
point cloud has 296 clusters


In [107]:
obbs = []
indexes = pd.Series(range(len(labels))).groupby(labels,sort=False).apply(list).tolist()
print(len(indexes))

297


In [110]:
Max_points = 1000
Min_points =  10
clusters = []
for i in range(0,len(indexes)):
    nb_pts = len(outlier_cloud.select_by_index(indexes[i]).points)
    if(nb_pts > Min_points and nb_pts < Max_points):
        sub_cloud = outlier_cloud.select_by_index(indexes[i])
        obb = sub_cloud.get_axis_aligned_bounding_box()
        obb.color=(0,0,1)
        obbs.append(obb)

In [ ]:
list_of_visuals=[]
list_of_visuals.append(outlier_cloud)
list_of_visuals.extend(obbs)
list_of_visuals.append(inlier_cloud)
open3d.visualization.draw_geometries(list_of_visuals)